In [8]:
! pip install sklearn
! pip install pandas
! pip install tqdm
! pip install -U sentence-transformers
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\adamm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement sentence_transformers==4.37.2 (from versions: 0.1.0, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.4.1, 0.2.5, 0.2.5.1, 0.2.6, 0.2.6.1, 0.2.6.2, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.5, 0.3.5.1, 0.3.6, 0.3.7, 0.3.7.1, 0.3.7.2, 0.3.8, 0.3.9, 0.4.0, 0.4.1, 0.4.1.1, 0.4.1.2, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.1.0, 1.1.1, 1.2.0, 1.2.1, 2.0.0, 2.1.0, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.5.0, 2.5.1, 2.6.0, 2.6.1, 2.7.0)
ERROR: No matching distribution found for sentence_transformers==4.37.2

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\adamm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --up


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\adamm\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [35]:
import pandas as pd
import torch
import random

from sentence_transformers import SentenceTransformer, losses, models, evaluation
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from torch import nn
from sklearn.metrics import matthews_corrcoef, f1_score, classification_report

from numpy.linalg import norm
import numpy as np
from tqdm import tqdm

Create drug-pairs dataset

In [2]:
df = pd.read_json("processed.json")
df.drop(columns=['DRUG_CID', "DRUG_NAME"], inplace=True)
df['INDEX'] = df.index
df1 = df.copy()
df2 = df.copy()
df1.columns = ['DRUG_ID1', 'INTERACTIONS1', 'STRUCTURE1', 'TARGET1', 'ENZYME1', 'PATH1', 'SIDE_EFFECTS1', 'INDICATIONS1', 'OFFSIDE_EFFECT1', 'INDEX1']
df2.columns = ['DRUG_ID2', 'INTERACTIONS2', 'STRUCTURE2', 'TARGET2', 'ENZYME2', 'PATH2', 'SIDE_EFFECTS2', 'INDICATIONS2', 'OFFSIDE_EFFECT2', 'INDEX2']
df = pd.merge(df1, df2, how="cross")
df = df[df['DRUG_ID1'] != df['DRUG_ID2']]
df.reset_index(inplace=True)
df['INTERACTION'] = df.apply(lambda row: row['INTERACTIONS1'][row['INDEX2']], axis=1)

df['id_pair'] = df[['DRUG_ID1', 'DRUG_ID2']].apply(lambda x: '-'.join(sorted(map(str, x))), axis=1)
df.drop_duplicates(subset='id_pair', inplace=True)
df.dropna(inplace=True)
df.drop(columns=["index", 'INTERACTIONS1', 'INTERACTIONS2', 'INDEX1', 'INDEX2', 'id_pair'], inplace=True)

In [3]:
data = {}
to_edit = ['STRUCTURE', 'TARGET', 'ENZYME', 'PATH', 'SIDE_EFFECTS', 'INDICATIONS', 'OFFSIDE_EFFECT']
for column in to_edit:
    column1 = column + '1'
    column2 = column + '2'
    data[column] = [InputExample(texts=[" ".join(map(str, row[0])), " ".join(map(str, row[1]))], label=row[2])
                    for row in zip(df[column1], df[column2], df['INTERACTION'])]

In [14]:
random.shuffle(data['STRUCTURE'])
split = int(len(data['STRUCTURE']) * 0.8)
train_data = data['STRUCTURE'][:split]
test_data = data['STRUCTURE'][split:]


In [45]:

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(train_data, batch_size=2)
train_loss = losses.ContrastiveLoss(model=model)

model.fit([(train_dataloader, train_loss)],epochs=10)
model.save("structure_model")


In [46]:
texts = [example.texts for example in test_data]
labels = [example.label for example in test_data]
encodings = [model.encode(text, convert_to_numpy=True, convert_to_tensor=False) for text in tqdm(texts)] 

100%|██████████| 19847/19847 [04:11<00:00, 79.05it/s]


In [51]:
predictions = [1 if np.dot(e[0], e[1]) / (norm(e[0]) * norm(e[1])) > 0.7 else 0 for e in encodings]

matthews = matthews_corrcoef(labels, predictions)
print("Matthews Correlation Coefficient:", matthews)
print(classification_report(labels, predictions))
len(train_data)

Matthews Correlation Coefficient: 0.7740839082548105
              precision    recall  f1-score   support

           0       0.92      0.94      0.93     13373
           1       0.86      0.83      0.85      6474

    accuracy                           0.90     19847
   macro avg       0.89      0.88      0.89     19847
weighted avg       0.90      0.90      0.90     19847



79388

In [ ]:
random.shuffle(data['TARGET'])
split = int(len(data['TARGET']) * 0.8)
train_data = data['TARGET'][:split]
test_data = data['TARGET'][split:]

In [ ]:
word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=64)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=64,
    activation_function=nn.Tanh(),
)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
train_dataloader = DataLoader(train_data, batch_size=2)
train_loss = losses.ContrastiveLoss(model=model)

model.fit([(train_dataloader, train_loss)],epochs=10)
model.save("target_model")
